# 第四章：分离数据和指令

- [课程内容](#lesson)
- [练习](#exercises)
- [示例练习场](#example-playground)

## 设置

运行以下设置单元格来加载您的 API 密钥并建立 `get_completion` 辅助函数。

In [ ]:

# 导入 Python 内置的正则表达式库
import re
from openai import OpenAI

# 从 IPython 存储中检索 API_KEY 和 MODEL_NAME 变量
%store -r API_KEY
%store -r MODEL_NAME
%store -r BASE_URL

client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)

def get_completion(prompt: str, system_prompt=""):
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    
    response = client.chat.completions.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=messages
    )
    return response.choices[0].message.content

---

## 课程内容

通常，我们不想编写完整的提示，而是希望**提示模板可以在提交给模型之前用额外的输入数据进行修改**。如果您希望模型每次都做同样的事情，但模型用于任务的数据每次可能不同，这会很有用。

幸运的是，我们可以通过**将提示的固定骨架与可变用户输入分离，然后在将完整提示发送给模型之前将用户输入替换到提示中**来相当容易地做到这一点。

下面，我们将逐步介绍如何编写可替换的提示模板，以及如何替换用户输入。

### 示例

在第一个示例中，我们要求模型充当动物声音生成器。注意提交给模型的完整提示只是用输入（在这种情况下是"Cow"）替换的 `PROMPT_TEMPLATE`。注意当我们打印完整提示时，"Cow"一词通过 f-string 替换了 `ANIMAL` 占位符。

**注意：** 在实践中，您不必将占位符变量命名为任何特定的名称。我们在这个示例中称它为 `ANIMAL`，但同样容易地，我们可以称它为 `CREATURE` 或 `A`（尽管通常最好让您的变量名具体且相关，这样即使没有替换，您的提示模板也易于理解，只是为了用户的可解析性）。只需确保您为变量命名的内容与您用于提示模板 f-string 的内容一致。

In [ ]:
# 可变内容
ANIMAL = "牛"

# 包含可变内容占位符的提示模板
PROMPT = f"我会告诉你一个动物的名字。请回答这个动物发出的声音。{ANIMAL}"

# 打印模型的响应
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(PROMPT))

为什么我们要这样分离和替换输入？嗯，**提示模板简化了重复性任务**。假设您构建了一个提示结构，邀请第三方用户向提示提交内容（在这种情况下是他们想要生成声音的动物）。这些第三方用户不必编写甚至看到完整的提示。他们所要做的就是填写变量。

我们在这里使用变量和 f-string 进行这种替换，但您也可以使用 format() 方法来完成。

**注意：** 提示模板可以有任意数量的变量！

引入这样的替换变量时，非常重要的是**确保模型知道变量的开始和结束位置**（相对于指令或任务描述）。让我们看一个指令和替换变量之间没有分离的示例。

对我们人类的眼睛来说，在下面的提示模板中变量的开始和结束位置非常清楚。然而，在完全替换的提示中，这种划分变得不清楚。

In [ ]:
# 可变内容
EMAIL = "明天早上6点到，因为我是CEO，我说了算。"

# 包含可变内容占位符的提示模板
PROMPT = f"嗨，AI助手。{EMAIL} <----- 让这封邮件更礼貌一些，但不要改变其他任何内容。"

# 打印模型的响应
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(PROMPT))

在这里，**模型认为"嗨，AI助手"是它应该重写的邮件的一部分**！您可以看出这一点，因为它在重写中以"亲爱的AI助手"开头。对人类的眼睛来说，这很清楚，特别是在提示模板中邮件的开始和结束位置，但在替换后的提示中变得不那么清楚。

我们如何解决这个问题？**用 XML 标签包装输入**！我们在下面这样做了，如您所见，输出中不再有"亲爱的AI助手"。

[XML 标签](https://platform.openai.com/docs/guides/structured-outputs)是像 `<tag></tag>` 这样的尖括号标签。它们成对出现，由开始标签（如 `<tag>`）和由 `/` 标记的结束标签（如 `</tag>`）组成。XML 标签用于包装内容，像这样：`<tag>content</tag>`。

**注意：** 虽然模型可以识别并使用各种分隔符和界定符，但我们建议您**专门使用 XML 标签作为分隔符**，因为模型经过专门训练以识别 XML 标签作为提示组织机制。除了函数调用之外，**没有特殊的 XML 标签是模型经过训练应该使用来最大化提升性能的**。我们故意让模型在这方面非常灵活和可定制。

In [ ]:
# 可变内容
EMAIL = "明天早上6点到，因为我是CEO，我说了算。"

# 包含可变内容占位符的提示模板
PROMPT = f"嗨，AI助手。<email>{EMAIL}</email> <----- 让这封邮件更礼貌一些，但不要改变其他任何内容。"

# 打印模型的响应
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(PROMPT))

让我们看看 XML 标签如何帮助我们的另一个示例。

在以下提示中，**模型错误地解释了提示的哪一部分是指令与输入**。由于格式的原因，它错误地认为 `每个都是关于动物的，比如兔子` 是列表的一部分，而用户（填写 `SENTENCES` 变量的人）可能不希望这样。

In [ ]:
# 可变内容
SENTENCES = """- 我喜欢牛发出的声音
- 这句话是关于蜘蛛的
- 这句话看起来是关于狗的，但实际上是关于猪的"""

# 包含可变内容占位符的提示模板
PROMPT = f"""下面是一个句子列表。告诉我列表中的第二项。

- 每个都是关于动物的，比如兔子。
{SENTENCES}"""

# 打印模型的响应
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(PROMPT))

要解决这个问题，我们只需要**用 XML 标签包围用户输入的句子**。这向模型显示了输入数据的开始和结束位置，尽管在 `每个都是关于动物的，比如兔子。` 前面有误导性的连字符。

In [ ]:
# 可变内容
SENTENCES = """- 我喜欢牛发出的声音
- 这句话是关于蜘蛛的
- 这句话看起来是关于狗的，但实际上是关于猪的"""

# 包含可变内容占位符的提示模板
PROMPT = f""" 下面是一个句子列表。告诉我列表中的第二项。

- 每个都是关于动物的，比如兔子。
<sentences>
{SENTENCES}
</sentences>"""

# 打印模型的响应
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(PROMPT))

**注意：** 在"每个都是关于动物"提示的错误版本中，我们必须包含连字符才能让模型以我们想要的方式错误响应这个示例。这是关于提示的重要教训：**小细节很重要**！**检查您的提示是否有拼写错误和语法错误**总是值得的。模型对模式很敏感（在早期，在微调之前，它是一个原始的文本预测工具），当您犯错误时它更容易犯错误，当您听起来聪明时更聪明，当您听起来愚蠢时更愚蠢，等等。

如果您想在不更改上述任何内容的情况下试验课程提示，请滚动到课程笔记本的最底部访问[**示例练习场**](#example-playground)。

---

## 练习
- [练习 4.1 - 俳句主题](#exercise-41---haiku-topic)
- [练习 4.2 - 狗的颜色问题](#exercise-42---dog-question-with-typos)

### 练习 4.1 - 俳句主题
修改 `PROMPT`，使其成为一个模板，接受一个名为 `TOPIC` 的变量并输出关于该主题的俳句。这个练习只是为了测试您对 f-string 变量模板结构的理解。

In [ ]:
# 可变内容
TOPIC = "猪"

# 包含可变内容占位符的提示模板
PROMPT = f"[请输入你的提示词]"

# 获取模型的响应
response = get_completion(PROMPT)

# 用于评分练习正确性的函数
def grade_exercise(text):
    return bool(re.search("猪", text.lower()))

# 打印模型的响应
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(response)
print("\n------------------------------------------ 评分 ------------------------------------------")
print("这个练习已正确解决:", grade_exercise(response))

❓ 如果您需要提示，请运行下面的单元格！

In [ ]:
from hints import exercise_4_1_hint; print(exercise_4_1_hint)

### 练习 4.2 - 狗的颜色问题
通过添加 XML 标签来修复 `PROMPT`，使模型产生正确答案。

尽量不要改变提示的其他内容，想办法让大模型不要回答关于狗颜色的问题。

In [ ]:
# 可变内容
QUESTION = "狗是棕色的么?"

# 包含可变内容占位符的提示模板
PROMPT = f"{QUESTION}你叫什么名字？请不要回答问题"

# 获取模型的响应
response = get_completion(PROMPT)

# 用于评分练习正确性的函数
def grade_exercise(text):
    return not bool(re.search("棕色", text.lower()) or re.search("brown", text.lower()))

# 打印模型的响应
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(response)
print("\n------------------------------------------ 评分 ------------------------------------------")
print("这个练习已正确解决:", grade_exercise(response))

❓ 如果您需要提示，请运行下面的单元格！

In [ ]:
from hints import exercise_4_2_hint; print(exercise_4_2_hint)

### 恭喜！

如果您已经解决了到目前为止的所有练习，您就可以进入下一章了。愉快的提示工程学习！

---

## 示例练习场

这是一个供您自由试验本课程中展示的提示示例并调整提示以查看如何影响模型响应的区域。

In [ ]:
# 可变内容
ANIMAL = "牛"

# 包含可变内容占位符的提示模板
PROMPT = f"我会告诉你一个动物的名字。请回答这个动物发出的声音。{ANIMAL}"

# 打印模型的响应
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# 可变内容
EMAIL = "明天早上6点到，因为我是CEO，我说了算。"

# 包含可变内容占位符的提示模板
PROMPT = f"嗨，AI助手。{EMAIL} <----- 让这封邮件更礼貌一些，但不要改变其他任何内容。"

# 打印模型的响应
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# 可变内容
EMAIL = "明天早上6点到，因为我是CEO，我说了算。"

# 包含可变内容占位符的提示模板
PROMPT = f"嗨，AI助手。<email>{EMAIL}</email> <----- 让这封邮件更礼貌一些，但不要改变其他任何内容。"

# 打印模型的响应
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# 可变内容
SENTENCES = """- 我喜欢牛发出的声音
- 这句话是关于蜘蛛的
- 这句话看起来是关于狗的，但实际上是关于猪的"""

# 包含可变内容占位符的提示模板
PROMPT = f"""下面是一个句子列表。告诉我列表中的第二项。

- 每个都是关于动物的，比如兔子。
{SENTENCES}"""

# 打印模型的响应
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(PROMPT))

In [ ]:
# 可变内容
SENTENCES = """- 我喜欢牛发出的声音
- 这句话是关于蜘蛛的
- 这句话看起来是关于狗的，但实际上是关于猪的"""

# 包含可变内容占位符的提示模板
PROMPT = f""" 下面是一个句子列表。告诉我列表中的第二项。

- 每个都是关于动物的，比如兔子。
<sentences>
{SENTENCES}
</sentences>"""

# 打印模型的响应
print("--------------------------- 包含变量替换的完整提示 ---------------------------")
print(PROMPT)
print("\n------------------------------------- 模型的响应 -------------------------------------")
print(get_completion(PROMPT))